In [ ]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector  # type: ignore
from datetime import datetime


connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='********',
    database='restaurant_booking'
)

if connection.is_connected():
    print("Connected to MySQL")

cursor = connection.cursor()


cursor.execute('''CREATE TABLE IF NOT EXISTS bookings
             (id INT PRIMARY KEY AUTO_INCREMENT, 
              name VARCHAR(255), 
              phone VARCHAR(20), 
              email VARCHAR(255), 
              date DATE, 
              time TIME, 
              guests INT)''')

connection.commit()

def add_booking():
    name = name_entry.get()
    phone = phone_entry.get()
    email = email_entry.get()
    date = date_entry.get()
    time = time_entry.get()
    guests = guests_entry.get()
    
    if not name or not phone or not email or not date or not time or not guests:
        messagebox.showerror("Input Error", "All fields are required!")
        return
    
    try:
        guests = int(guests)
    except ValueError:
        messagebox.showerror("Input Error", "Guests must be a number!")
        return
    
    if not validate_date(date):
        messagebox.showerror("Date Error", "Invalid Date format! Please use YYYY-MM-DD.")
        return
    if not validate_time(time):
        messagebox.showerror("Time Error", "Invalid Time format! Please use HH:MM (24-hour format).")
        return
        
    cursor.execute("INSERT INTO bookings (name, phone, email, date, time, guests) VALUES (%s, %s, %s, %s, %s, %s)", (name, phone, email, date, time, guests))
    connection.commit()

    messagebox.showinfo("Booking Added", f"Booking added for {name}!")
    clear_fields()
    update_queue_display()

def remove_booking():
    cursor.execute("SELECT * FROM bookings ORDER BY id ASC LIMIT 1")
    booking = cursor.fetchone()

    if booking:
        
        cursor.execute("DELETE FROM bookings WHERE id = %s", (booking[0],))
        connection.commit()
        
        messagebox.showinfo("Booking Removed", f"Booking for {booking[1]} on {booking[4]} at {booking[5]} has been removed.")
        update_queue_display()
    else:
        messagebox.showerror("Queue Empty", "No bookings to remove.")

def validate_date(date):
    try:
        datetime.strptime(date, "%Y-%m-%d")
        return True
    except ValueError:
        return False

def validate_time(time):
    try:
        datetime.strptime(time, "%H:%M")
        return True
    except ValueError:
        return False

def update_queue_display():
    queue_list.delete(0, tk.END)
    cursor.execute("SELECT * FROM bookings")
    bookings = cursor.fetchall()
    
    for booking in bookings:
        queue_list.insert(tk.END, f"{booking[1]} - {booking[4]} {booking[5]}, {booking[6]} guests")

def clear_fields():
    name_entry.delete(0, tk.END)
    phone_entry.delete(0, tk.END)
    email_entry.delete(0, tk.END)
    date_entry.delete(0, tk.END)
    time_entry.delete(0, tk.END)
    guests_entry.delete(0, tk.END)

root = tk.Tk()
root.title("Restaurant Booking System")

root.state('zoomed')

root.configure(bg='#f0f0f0')

title_label = tk.Label(root, text="Restaurant Booking System", font=("Arial", 24, "bold"), bg='#3e64ff', fg='white')
title_label.pack(fill=tk.X, pady=20)

frame = tk.Frame(root, bg='#f0f0f0')
frame.pack(pady=50)

tk.Label(frame, text="Name:", font=("Arial", 14), bg='#f0f0f0').grid(row=0, column=0, padx=10, pady=10, sticky="e")
tk.Label(frame, text="Phone:", font=("Arial", 14), bg='#f0f0f0').grid(row=1, column=0, padx=10, pady=10, sticky="e")
tk.Label(frame, text="Email:", font=("Arial", 14), bg='#f0f0f0').grid(row=2, column=0, padx=10, pady=10, sticky="e")
tk.Label(frame, text="Date (YYYY-MM-DD):", font=("Arial", 14), bg='#f0f0f0').grid(row=3, column=0, padx=10, pady=10, sticky="e")
tk.Label(frame, text="Time (HH:MM):", font=("Arial", 14), bg='#f0f0f0').grid(row=4, column=0, padx=10, pady=10, sticky="e")
tk.Label(frame, text="Guests:", font=("Arial", 14), bg='#f0f0f0').grid(row=5, column=0, padx=10, pady=10, sticky="e")

name_entry = tk.Entry(frame, font=("Arial", 12))
name_entry.grid(row=0, column=1, padx=10, pady=10, sticky="w")

phone_entry = tk.Entry(frame, font=("Arial", 12))
phone_entry.grid(row=1, column=1, padx=10, pady=10, sticky="w")

email_entry = tk.Entry(frame, font=("Arial", 12))
email_entry.grid(row=2, column=1, padx=10, pady=10, sticky="w")

date_entry = tk.Entry(frame, font=("Arial", 12))
date_entry.grid(row=3, column=1, padx=10, pady=10, sticky="w")

time_entry = tk.Entry(frame, font=("Arial", 12))
time_entry.grid(row=4, column=1, padx=10, pady=10, sticky="w")

guests_entry = tk.Entry(frame, font=("Arial", 12))
guests_entry.grid(row=5, column=1, padx=10, pady=10, sticky="w")

add_button = tk.Button(frame, text="Add Booking", command=add_booking, font=("Arial", 12), bg="#4CAF50", fg="white", padx=20, pady=10)
add_button.grid(row=6, column=0, pady=20, padx=10)

remove_button = tk.Button(frame, text="Remove Booking", command=remove_booking, font=("Arial", 12), bg="#f44336", fg="white", padx=20, pady=10)
remove_button.grid(row=6, column=1, pady=20, padx=10)

queue_list = tk.Listbox(frame, width=50, font=("Arial", 12), height=10)
queue_list.grid(row=7, column=0, columnspan=2, pady=10, padx=10)

scrollbar = tk.Scrollbar(frame)
scrollbar.grid(row=7, column=2, sticky='ns')

queue_list.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=queue_list.yview)

update_queue_display()

root.mainloop()